# NLP

## NLTK

We got a column with a new extract summarizing the incident. Let's do Natural Language Process so the text is tokenized and we can find the sentiment.

Furthermore, this will allow us to create a new feature in the global terrorist DataFrame so it provides more information to build the Clusters in the Unsupervised Learning.

In [1]:
import pandas as pd
import numpy as np

import re

import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.probability import FreqDist


In [2]:
gt = pd.read_csv('CSVs/globalterrorism.csv')
display(gt.shape)
gt.head()

(43793, 19)

,iyear,imonth,iday,more_than_24h,country_txt,city,latitude,longitude,news,suicide,attacktype,attacktype_txt,targtype,targtype_txt,band_name,npeople_attacking,weapon,weapon_txt,nkill
0,1970,3,5,0,United States,San Juan,18.386932,-66.061127,3/5/1970: Unknown Puerto Rican Nationalists as...,0,1,Assassination,4,Military,Other,1,5,Firearms,2
1,1970,3,9,0,United States,Bel Air,39.538940,-76.348479,3/9/1970: A bomb exploded in the car of Ralph ...,0,3,Bombing/Explosion,20,Unknown,Other,2,6,Explosives,2
2,1970,3,20,0,United States,Detroit,42.331685,-83.047924,3/20/1970: Unknown perpetrators shot and kille...,0,1,Assassination,14,Private Citizens & Property,Other,7,5,Firearms,1
3,1970,4,24,0,United States,Baltimore,39.308342,-76.616104,4/24/1970: Three members of the Black Panthers...,0,2,Armed Assault,3,Police,Other,3,5,Firearms,1
4,1970,5,9,0,United States,Sacramento,38.551295,-121.471066,5/9/1970: Sacramento Police Officer Bernard Be...,0,2,Armed Assault,3,Police,Other,40,5,Firearms,1


In [3]:
gt.news.value_counts()

02/09/2015: Assailants attacked residents and buildings in Logo district, Benue state, Nigeria. This was one of 24 coordinated raids on villages and communities in this area on February 9, 2015. At least 18 people were killed across attacks. No group claimed responsibility for the incident; however, sources attributed the attack to Fulani militants.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             17
05/03/2015: Assailants attacked a police checkpoint in Warduj district, Badakhshan Province, Afghanistan.

In [4]:
def SentimentAnalysis(df, process_column="text", n_common=5000):
    
    def clean_up(s):
        s = re.sub("\d{2}/\d{2}/\d{4}:", ' ', s) #clean dates
        s = re.sub('\d+', ' ', s) # /d Any numeric character
        s = re.sub('\W+', ' ', s) # Any non-alphanumeric character
        return s.lower().strip()

    def tokenize(s):
        return word_tokenize(clean_up(s))
    
    def stem_and_lemmatize(s):
        return [WordNetLemmatizer().lemmatize(SnowballStemmer("english").stem(x)) for x in tokenize(s)]

    def remove_stopwords(s):
        return [w for w in stem_and_lemmatize(s) if not w in stopwords.words('english')]
    
    def find_features(document):
        top = [x[0] for x in FreqDist(lista).most_common(n_common)]
        words = set(document)
        features = {}
        for w in top:
            features[w] = (w in words)
        s = SentimentIntensityAnalyzer().polarity_scores(" ".join(document))
        if s["pos"] > 0.2:
            s = True
        else:
            s = False
        return (features, s)
    
    lista = []
   
    df['news_processed'] = df[process_column].apply(remove_stopwords)
    for x in df['news_processed']:
        lista += x
    feature = list(df['news_processed'].apply(find_features))
    train_set, test_set = feature[5000:], feature[:5000]
    cls = nltk.NaiveBayesClassifier.train(train_set)
    
    return "Classifier accuracy percent:",(nltk.classify.accuracy(classifier, test_set))*100

In [ ]:
print(SentimentAnalysis(df=gt, process_column="news"))